In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user\\Desktop\\poultry_diseases\\Chicken-Disease-Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\user\\Desktop\\poultry_diseases\\Chicken-Disease-Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : str    

In [6]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


In [7]:
import os
import urllib.request as request
import zipfile
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path : str
        Extract the zip file into data directory
        Function return none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-01-29 19:01:49,254 INFO common yaml file: config\config.yaml loaded successfully]
[2024-01-29 19:01:49,276 INFO common yaml file: params.yaml loaded successfully]
[2024-01-29 19:01:49,280 INFO common created directory at: artifacts]
[2024-01-29 19:01:49,288 INFO common created directory at: artifacts/data_ingestion]


[2024-01-29 19:02:33,225 INFO 404859855 artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B4B6:30D29B:179875:259A98:65B7A8E5
Accept-Ranges: bytes
Date: Mon, 29 Jan 2024 13:32:21 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100032-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1706535141.215836,VS0,VE710
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: e63c0ccff5f1599c8b723df9237f860bb20a21d8
Expires: Mon, 29 Jan 2024 13:37:21 GMT
Source-Age: 0

]
